In [3]:
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
import branca

print(folium.__version__)

0.11.0


# Interactive Choropleth Map using Folium and Python

Cudos to:

Medium: https://medium.com/@lindsayrallen1/interactive-choropleth-map-of-washington-dc-using-folium-and-python-2794708514d5

Github: https://github.com/lindsallen/mapping/blob/master/dc_choropleth_wtooltip.ipynb

### Folium.docs 
    https://python-visualization.github.io/folium/index.html

First part taken from: 

### Interactive choropleth with Python and Folium (and some tips)

https://vverde.github.io/blob/interactivechoropleth.html

In [7]:
# Read geodata nrw_gemeinden from data folder

fname = './data/nrw_gemeinden.json'
map = gpd.read_file(fname)
map.head()

,id,AGS,GEN,cx,cy,geometry
0,None,05162022,Meerbusch,6.670602,51.283494,"POLYGON ((6.69490 51.23471, 6.68511 51.23396, ..."
1,None,05162024,Neuss,6.688130,51.180270,"POLYGON ((6.67634 51.23247, 6.68511 51.23396, ..."
2,None,05162028,Rommerskirchen,6.686148,51.052101,"POLYGON ((6.69636 51.10123, 6.69670 51.09872, ..."
3,None,05166004,Brüggen,6.163028,51.270354,"POLYGON ((6.24154 51.25280, 6.23760 51.24656, ..."
4,None,05166008,Grefrath,6.345855,51.347348,"POLYGON ((6.38632 51.33133, 6.38992 51.32733, ..."


In [8]:
map=map[['AGS', 'GEN', 'geometry']]
map.head()

,AGS,GEN,geometry
0,05162022,Meerbusch,"POLYGON ((6.69490 51.23471, 6.68511 51.23396, ..."
1,05162024,Neuss,"POLYGON ((6.67634 51.23247, 6.68511 51.23396, ..."
2,05162028,Rommerskirchen,"POLYGON ((6.69636 51.10123, 6.69670 51.09872, ..."
3,05166004,Brüggen,"POLYGON ((6.24154 51.25280, 6.23760 51.24656, ..."
4,05166008,Grefrath,"POLYGON ((6.38632 51.33133, 6.38992 51.32733, ..."


In [9]:
map.crs = "EPSG:4326"

In [11]:
population = pd.read_csv('./data/durchschnittsalter_nrw_gemeinden.csv', dtype={'AGS': 'str'}, decimal=',')
population.head(5)

,Unnamed: 0,Jahr,AGS,Ort,alter,männlich,weiblich
0,2722,31.12.2018,05974008,Bad Sassendorf,49.7,47.5,51.7
1,2404,31.12.2018,05358012,"Heimbach, Eifel",48.4,47.1,49.8
2,2366,31.12.2018,05170016,Hünxe,48.3,46.9,49.6
3,2429,31.12.2018,05366004,Bad Münstereifel,47.8,46.5,49.0
4,2666,31.12.2018,05954020,Herdecke,47.6,46.1,49.0


In [12]:
population=population[['AGS','alter']]
population.head(5)

,AGS,alter
0,05974008,49.7
1,05358012,48.4
2,05170016,48.3
3,05366004,47.8
4,05954020,47.6


In [13]:
mappop=map.merge(population,on="AGS")
mappop.head()

,AGS,GEN,geometry,alter
0,05162022,Meerbusch,"POLYGON ((6.69490 51.23471, 6.68511 51.23396, ...",45.8
1,05162024,Neuss,"POLYGON ((6.67634 51.23247, 6.68511 51.23396, ...",43.9
2,05162028,Rommerskirchen,"POLYGON ((6.69636 51.10123, 6.69670 51.09872, ...",44.3
3,05166004,Brüggen,"POLYGON ((6.24154 51.25280, 6.23760 51.24656, ...",46.2
4,05166008,Grefrath,"POLYGON ((6.38632 51.33133, 6.38992 51.32733, ...",46.7


In [14]:
mappop.columns

Index(['AGS', 'GEN', 'geometry', 'alter'], dtype='object')

In [15]:
x_map=map.centroid.x.mean()
y_map=map.centroid.y.mean()
print(x_map,y_map)

7.4600288713908345 51.456367823477706


/home/p3nny/pfv/myvenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/home/p3nny/pfv/myvenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [16]:
m = folium.Map(location=[y_map, x_map], zoom_start=8,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
m

### folium.Choropleth

### folium.GeoJSON

In [17]:
variable = 'alter'
name = 'Durchschnittsalter'
print(name, 'colorscale')

print('Min:', mappop.alter.min())
print('Max:', mappop.alter.max())

Durchschnittsalter colorscale
Min: 38.7
Max: 49.7


In [18]:
df = mappop[['GEN', variable]].sort_values(by=variable, ascending=False)
df.reset_index(inplace=True)
quantiles = list(df[df.index.isin([0,66, 132, 198, 264, 330])][variable])
quantiles.append(0)
quantiles.sort()
quantiles

[0, 43.4, 44.1, 44.7, 45.2, 46.0, 49.7]

In [19]:
colorscale = branca.colormap.linear.YlOrRd_09.scale(mappop.alter.min(), mappop.alter.max())
colorscale = colorscale.to_step(n=6, quantiles=quantiles)
colorscale.caption = name ## adds name for legend
colorscale

In [20]:
folium.GeoJson(mappop, ## GeoPandas dataframe
               name="Durchschnittsalter",
                   
               ## controls the fill of the geo regions; applying colorscale based on variable
               style_function=lambda x: {"weight":1
                                         , 'color': '#545453'
                                         ## if variable is 0 map is a very light grey
                                         ## else colorscale applies based on variable
                                         , 'fillColor':'#9B9B9B' if x['properties'][variable] == 0 
                                         else colorscale(x['properties'][variable])
                                         ## similarly opacity is increased if value is 0
                                         , 'fillOpacity': 0.2 if x['properties'][variable] == 0 
                                         else 0.5},
                   
               ## changes styling of geo regions upon hover
               highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
               
                ## tooltip can include information from any column in the GeoPandas dataframe   
                tooltip=folium.features.GeoJsonTooltip(
                fields=['GEN', variable],
                aliases=['Ort:', name])
              ).add_to(m)

## add colorscale to map so that it appears as the legend
colorscale.add_to(m)
    
m